In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!git clone https://github.com/semajyllek/ctmatch.git

Cloning into 'ctmatch'...
remote: Enumerating objects: 1522, done.
remote: Counting objects: 100% (472/472), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 1522 (delta 327), reused 392 (delta 248), pack-reused 1050
Receiving objects: 100% (1522/1522), 414.13 KiB | 16.56 MiB/s, done.
Resolving deltas: 100% (1011/1011), done.


In [3]:
!pip install -r /content/ctmatch/requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 11.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 97.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 50.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5

In [4]:
import sys
sys.path.insert(1, '/content/ctmatch');sys.path

['/content',
 '/content/ctmatch',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [5]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
from pathlib import Path
TREC_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/trec_data.jsonl')
KZ_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/kz_data.jsonl')
COMBINED_DATA_PATH = Path('/content/drive/MyDrive/ct_data23/combined_classifier_data.jsonl')

In [7]:
from ctmatch.match import ModelConfig, CTMatch

In [8]:
from transformers import logging
logging.set_verbosity_error()

In [9]:
nli_model = 'cross-encoder/nli-roberta-base'
scibert_model = 'allenai/scibert_scivocab_uncased'
biogpt_model = 'microsoft/biogpt'
bart_model = 'facebook/bart-large'
gpt2_model = 'gpt2'
finetuned_scibert = 'semaj83/scibert_finetuned_ctmatch'
model_name = finetuned_scibert


model_config = ModelConfig(
    name=f'ctmatch_finetuned_{model_name}-classifier',
    classifier_model_checkpoint=model_name,
    max_length=512,
    batch_size=8,
    padding='max_length',
    truncation=True,
    learning_rate=2e-5,
    train_epochs=6,
    weight_decay=0.01,
    warmup_steps=500,
    seed=42,
    splits={"train":0.8, "val":0.1},
    output_dir = f"/content/drive/MyDrive/finetuned_{model_name.split('/')[-1]}model",
    use_trainer=True,
    convert_snli=False,
    fp16=True,
    early_stopping=True,
    push_to_hub=False
)

ctm = CTMatch(model_config)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/semaj83___json/semaj83--ctmatch_classification-c5ae6b4cf5476186/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

Map:   0%|          | 0/28576 [00:00<?, ? examples/s]

Map:   0%|          | 0/7939 [00:00<?, ? examples/s]

Map:   0%|          | 0/3176 [00:00<?, ? examples/s]

In [10]:
ctm.classifier_model.dataset

DatasetDict({
    train: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 28576
    })
    test: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 7939
    })
    validation: Dataset({
        features: ['doc', 'labels', 'topic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3176
    })
})

In [11]:
ctm.classifier_model.dataset['train'].features

{'doc': Value(dtype='string', id=None),
 'labels': ClassLabel(names=['not_relevant', 'partially_relevant', 'relevant'], id=None),
 'topic': Value(dtype='string', id=None),
 'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None),
 'token_type_ids': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None),
 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}

In [12]:
ctm.classifier_model.tokenizer.decode(ctm.classifier_model.dataset['train'][0]['input_ids'])

"[CLS] a 22 - year - old caucasian man came to the clinic with a history of tremors since a year ago. the tremor was first in his right hand while holding something. later the tremor became continuous and extended to both hands and legs and even at rest. the kayser - fleischer'ring was detected in the ophthalmologic exam. the physical exam revealed jaundice, hepatosplenomegaly and hypotonia of the upper limbs. he had a constant smile on his face, however, he has aggressive behavior according to his parents'explanation. his laboratory study was significant for a low serum caeruloplasmin ( 0. 05 g / l ), and a raised 24 hour urine copper excretion ( 120 μg / 24 h ). wilson disease was confirmed by high liver copper concentration ( 305 μg / g dry weight of liver ). [SEP] eligible ages ( years ) : 18. 0 - 70. 0, efficacy and safety ( 1 ) 18 years < the age of patients > 70 years. ( 2 ) karnofsky performance scale ( kps ) ≥ 60. ( 2 ) histologically confirmed diagnosis of gbm world health or

In [13]:
import torch
torch.cuda.empty_cache()

In [14]:
#ctm.classifier_model.manual_train()

In [15]:
#ctm.classifier_model.train_and_predict()

In [16]:
cm, cr = ctm.classifier_model.get_sklearn_metrics()

In [17]:
print(cm)

[[5036  260  134]
 [ 410  742  179]
 [ 281  319  578]]


In [18]:
print(cr)

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      5430
           1       0.56      0.56      0.56      1331
           2       0.65      0.49      0.56      1178

    accuracy                           0.80      7939
   macro avg       0.70      0.66      0.67      7939
weighted avg       0.79      0.80      0.79      7939

